In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
import pylab as pl
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)

## Specifiy dementia subtype

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

## Specify exposure window

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

## Load relevant dataframe and create intercept

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_' + subtype + '_' + window +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [8]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,insomnia,stroke,non_stroke_vascular_disease,hypertension,diabetes,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,chronic_pulmonary_disease,epilepsy,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,other_sedatives_100_pdds,antipsychotics_100_pdds,depot_antipsychotics_100_pdds,antidepressants_100_pdds,non_insomnia_GP_consultations,benzo_and_z_drugs_any,insomnia_any,insomnia_count:0,insomnia_count:1_5,insomnia_count:6_10,insomnia_count:above_10,non_insomnia_GP_consultations:0,non_insomnia_GP_consultations:1_10,non_insomnia_GP_consultations:11_100,non_insomnia_GP_consultations:101_1000,non_insomnia_GP_consultations:above_1000,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,antidepressant_pdds:00000,antidepressant_pdds:00001_10,antidepressant_pdds:00011_100,antidepressant_pdds:00101_1000,antidepressant_pdds:01001_10000,antidepressant_pdds:10000_and_above,antidepressant_pdds:10000_and_above,antipsychotic_pdds:00000,antipsychotic_pdds:00001_10,antipsychotic_pdds:00011_100,antipsychotic_pdds:00101_1000,antipsychotic_pdds:01001_10000,antipsychotic_pdds:10000_and_above,antipsychotic_pdds:10000_and_above,depot_antipsychotic_pdds:00000,depot_antipsychotic_pdds:00001_10,depot_antipsychotic_pdds:00011_100,depot_antipsychotic_pdds:00101_1000,depot_antipsychotic_pdds:01001_10000,depot_antipsychotic_pdds:10000_and_above,depot_antipsychotic_pdds:10000_and_above,other_sedative_pdds:00000,other_sedative_pdds:00001_10,other_sedative_pdds:00011_100,other_sedative_pdds:00101_1000,other_sedative_pdds:01001_10000,other_sedative_pdds:10000_and_above,other_sedative_pdds:10000_and_above,benzo_and_z_drug_pdds:00000,benzo_and_z_drug_pdds:00001_10,benzo_and_z_drug_pdds:00011_100,benzo_and_z_drug_pdds:00101_1000,benzo_and_z_drug_pdds:01001_10000,benzo_and_z_drug_pdds:10000_and_above,benzo_and_z_drug_pdds:10000_and_above,mood_stabiliser_pdds:00000,mood_stabiliser_pdds:00001_10,mood_stabiliser_pdds:00011_100,mood_stabiliser_pdds:00101_1000,mood_stabiliser_pdds:01001_10000,mood_stabiliser_pdds:10000_and_above,mood_stabiliser_pdds:10000_and_above
803,272473,25,473,0,2011-03-03,True,6578.0,1999-12-16,2011-09-01,71204,86,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,98,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0
7944,2095655,32,655,1,2010-01-20,False,NaN,1999-12-10,2013-04-22,23611,78,0,0,0,1,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,214,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0
11026,7821415,23,415,1,2008-12-03,False,NaN,1998-07-17,2012-07-17,26948,85,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,67,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0
13807,5955194,33,194,1,2008-04-21,True,7664.0,1996-08-09,2012-01-05,29102,75,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,88,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0
8589,4542639,21,639,1,2009-08-24,False,NaN,1999-06-22,2012-11-26,13240,88,0,0,1,1,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,135,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0

In [9]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'insomnia', 'stroke',
       'non_stroke_vascular_disease', 'hypertension', 'diabetes',
       'mental_illness_non_smi', 'mental_illness_smi', 'sleep_apnoea',
       'chronic_pulmonary_disease', 'epilepsy', 'mood_stabilisers_100_pdds',
       'benzo_and_z_drugs_100_pdds', 'other_sedatives_100_pdds',
       'antipsychotics_100_pdds', 'depot_antipsychotics_100_pdds',
       'antidepressants_100_pdds', 'non_insomnia_GP_consultations',
       'benzo_and_z_drugs_any', 'insomnia_any', 'insomnia_count:0',
       'insomnia_count:1_5', 'insomnia_count:6_10', 'insomnia_count:above_10',
       'non_insomnia_GP_consultations:0', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000', 'age_at_index_date:65-69'

In [10]:
pt_features['intercept'] = 1.0

In [11]:
# pt_features.sort_values(by='insomnia_count<=10',ascending=False)

In [12]:
columns_always_for_inclusion = [
    'female',
    'non_insomnia_GP_consultations', 'stroke',
    'non_stroke_vascular_disease', 'hypertension',
    'diabetes', 'mental_illness_non_smi', 'mental_illness_smi',
    'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy',
    'mood_stabilisers_100_pdds',
    'other_sedatives_100_pdds', 'antidepressants_100_pdds', 
    'depot_antipsychotics_100_pdds', 'antipsychotics_100_pdds',
    'benzo_and_z_drugs_100_pdds'
]

## Insomnia consultations

### insomnia yes/no

In [13]:
cols_for_inclusion_here = ['insomnia_any']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

female  being retained as mean > 0
stroke  being retained as mean > 0
non_stroke_vascular_disease  being retained as mean > 0
hypertension  being retained as mean > 0
diabetes  being retained as mean > 0
mental_illness_non_smi  being retained as mean > 0
mental_illness_smi  being retained as mean > 0
sleep_apnoea  being retained as mean > 0
chronic_pulmonary_disease  being retained as mean > 0
epilepsy  being retained as mean > 0
mood_stabilisers_100_pdds  being retained as mean > 0
benzo_and_z_drugs_100_pdds  being retained as mean > 0
other_sedatives_100_pdds  being retained as mean > 0
antipsychotics_100_pdds  being retained as mean > 0
depot_antipsychotics_100_pdds  being retained as mean > 0
antidepressants_100_pdds  being retained as mean > 0
non_insomnia_GP_consultations  being retained as mean > 0
insomnia_any  being retained as mean > 0


*Stage 1*
Univariate results
                               Univariate OR  p value  [0.025  0.975]
antidepressants_100_pdds               1.

In [14]:
univariate_and_multivariate_results

,Univariate OR,p value,[0.025,0.975],Multivariate OR,p value,[0.025,0.975]
antidepressants_100_pdds,1.029,0.000,1.020,1.039,1.014,0.005,1.004,1.024
chronic_pulmonary_disease,1.026,0.483,0.956,1.100,0.910,0.022,0.840,0.986
epilepsy,1.688,0.000,1.306,2.182,1.454,0.005,1.120,1.887
female,1.000,1.000,0.962,1.039,0.873,0.000,0.826,0.922
hypertension,0.951,0.031,0.909,0.996,0.802,0.000,0.753,0.855
insomnia_any,1.219,0.000,1.120,1.327,1.118,0.018,1.019,1.226
mental_illness_non_smi,1.377,0.000,1.301,1.457,1.422,0.000,1.324,1.527
non_insomnia_GP_consultations,1.001,0.000,1.000,1.001,1.001,0.000,1.000,1.001
stroke,1.267,0.000,1.141,1.407,1.177,0.004,1.052,1.316


### insomnia (continuous variable)

In [15]:
cols_for_inclusion_here = ['insomnia','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

female  being retained as mean > 0
insomnia  being retained as mean > 0
stroke  being retained as mean > 0
non_stroke_vascular_disease  being retained as mean > 0
hypertension  being retained as mean > 0
diabetes  being retained as mean > 0
mental_illness_non_smi  being retained as mean > 0
mental_illness_smi  being retained as mean > 0
sleep_apnoea  being retained as mean > 0
chronic_pulmonary_disease  being retained as mean > 0
epilepsy  being retained as mean > 0
mood_stabilisers_100_pdds  being retained as mean > 0
benzo_and_z_drugs_100_pdds  being retained as mean > 0
other_sedatives_100_pdds  being retained as mean > 0
antipsychotics_100_pdds  being retained as mean > 0
depot_antipsychotics_100_pdds  being retained as mean > 0
antidepressants_100_pdds  being retained as mean > 0
non_insomnia_GP_consultations  being retained as mean > 0


*Stage 1*
Univariate results
                               Univariate OR  p value  [0.025  0.975]
antidepressants_100_pdds               1.029 

In [16]:
univariate_and_multivariate_results

,Univariate OR,p value,[0.025,0.975],Multivariate OR,p value,[0.025,0.975]
antidepressants_100_pdds,1.029,0.000,1.020,1.039,1.013,0.008,1.003,1.023
benzo_and_z_drugs_100_pdds,1.012,0.001,1.005,1.019,1.002,0.516,0.995,1.010
chronic_pulmonary_disease,1.026,0.483,0.956,1.100,0.912,0.025,0.842,0.989
epilepsy,1.688,0.000,1.306,2.182,1.456,0.005,1.122,1.890
female,1.000,1.000,0.962,1.039,0.876,0.000,0.829,0.925
hypertension,0.951,0.031,0.909,0.996,0.803,0.000,0.754,0.856
insomnia,1.053,0.004,1.017,1.090,1.015,0.402,0.980,1.052
mental_illness_non_smi,1.377,0.000,1.301,1.457,1.427,0.000,1.329,1.533
non_insomnia_GP_consultations,1.001,0.000,1.000,1.001,1.001,0.000,1.000,1.001
stroke,1.267,0.000,1.141,1.407,1.177,0.004,1.052,1.317


### insomnia quantiles

In [17]:
cols_for_inclusion_here = ['insomnia_count:1_5','insomnia_count:6_10','insomnia_count:above_10','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

female  being retained as mean > 0
stroke  being retained as mean > 0
non_stroke_vascular_disease  being retained as mean > 0
hypertension  being retained as mean > 0
diabetes  being retained as mean > 0
mental_illness_non_smi  being retained as mean > 0
mental_illness_smi  being retained as mean > 0
sleep_apnoea  being retained as mean > 0
chronic_pulmonary_disease  being retained as mean > 0
epilepsy  being retained as mean > 0
mood_stabilisers_100_pdds  being retained as mean > 0
benzo_and_z_drugs_100_pdds  being retained as mean > 0
other_sedatives_100_pdds  being retained as mean > 0
antipsychotics_100_pdds  being retained as mean > 0
depot_antipsychotics_100_pdds  being retained as mean > 0
antidepressants_100_pdds  being retained as mean > 0
non_insomnia_GP_consultations  being retained as mean > 0
insomnia_count:1_5  being retained as mean > 0
insomnia_count:6_10  being retained as mean > 0
insomnia_count:above_10  being retained as mean > 0


*Stage 1*
Univariate results
     

In [18]:
univariate_and_multivariate_results

,Univariate OR,p value,[0.025,0.975],Multivariate OR,p value,[0.025,0.975]
antidepressants_100_pdds,1.029,0.000,1.020,1.039,1.013,0.008,1.003,1.023
benzo_and_z_drugs_100_pdds,1.012,0.001,1.005,1.019,1.002,0.522,0.995,1.010
chronic_pulmonary_disease,1.026,0.483,0.956,1.100,0.909,0.020,0.839,0.985
epilepsy,1.688,0.000,1.306,2.182,1.450,0.005,1.117,1.883
female,1.000,1.000,0.962,1.039,0.871,0.000,0.824,0.921
hypertension,0.951,0.031,0.909,0.996,0.802,0.000,0.753,0.855
insomnia_count:1_5,1.220,0.000,1.120,1.329,1.120,0.017,1.021,1.230
insomnia_count:6_10,1.667,0.162,0.815,3.409,1.411,0.352,0.684,2.911
insomnia_count:above_10,0.556,0.292,0.186,1.658,0.382,0.089,0.127,1.156
mental_illness_non_smi,1.377,0.000,1.301,1.457,1.422,0.000,1.324,1.526
